# Context

In October 2021 the World Health Organization (WHO) published an article about how Mental Health services have been pushed to the limit during the COVID-19 pandemic while stating that the next pandemic will be on mental health encouraging governments to increase their expenditure on Mental Health.

While searching for Mental Health service usage and expenditure in New Zealand we found that the information is spread in several excel spreadsheets from 2002 to 2020 with different sources, formats, and accessibility. 

We then proceeded to gather some of the information provided by the New Zealand Ministry of health into 3 datasets that 
summarise the usage of these services in the last 20 years.

# Inspiration

The main inspiration for this dataset was to find a way of building a continuous pipeline for future reference of Mental Health Service usage in New Zealand.

Our first approach was to match Government Expenditure and usage of Mental Health Services in NZ over the last years but finding information about specific government expenditure is hard to come by, So we decided to focus mostly in creating a solid dataframe about mental health service usage over the years.


# Data Sources

[Ministry of Health New Zealand](https://www.health.govt.nz)

This data source was chosen for its ease of access and ability to web scrape.

Datasets were available from three sources:
1) Datasets from 2002 to 2008
2) Dataset of 2010
3) Dataset from 2011 to 2020

3 different crawlers were developed in order to maintain consistency over sources.
Datasets from 2011 onwards are displayed in the ministry of health new [Aggregated Data Site.](https://www.health.govt.nz/our-work/mental-health-and-addiction/mental-health-and-addiction-monitoring-reporting-and-data). 

Datasets from 2002 were gathered from legacy sources on the list of reports by the [ministry of health](https://www.health.govt.nz/nz-health-statistics/health-statistics-and-data-sets/mental-health-data-and-stats?mega=Health%20statistics&title=Mental%20health)


The data provided in this data sets can be classified into 3 groups:

1) **NZ Mental Health services usage by gender, age and ethnicity.**
2) **NZ Mental Health service usage by DHB's (District Health boards)**
3) **NZ Metal Health service usage by Service provided.**

"Data is sourced from the Programme for the Integration of Mental Health Data (PRIMHD). PRIMHD contains Ministry of Health funded mental health and addiction service activity and outcomes data. The data is collected from district health boards (DHBs) and non-governmental organisations (NGOs). 

PRIMHD data is used to report on what services are being provided, who is providing the services, and what outcomes are being achieved for health consumers across New Zealand's mental health sector. These reports enable better quality service planning and decision making by mental health and addiction service providers, at local, regional and national levels."											

We have combined the data in the excel files provided by the MOH into a single data frame.

# Intention of usage.

The idea behind the project is to have an incremental dataset for past and future reference, allow ease of access to timeseries information and better visibility.

# Problems while gathering data.

We will divide the problems while gathering information into 2 categories: Downloading and Wrangling.

## Downloading

The ministry of health releases an anual report on Mental Health since 2002, this reports are uploaded to the stats page of the Ministry of Health's website.

Reports from 2002 to 2007 have an aggregated site where they can be downlaoded programmatically. 
Reports from 2008 and 2010 have their individual site and had to be added manually to the download process.
Reports from 2011 onwards have their own Mental Health page where they get uploaded every year. This website allows us to make incremental updates to the current dataset.

This 3 types of published papers required individual processes to download programmatically, 2 of them were scrapped from lists, and 1 of them manually added to the dataset.

While developing the download process for reports from 2011 we also find that some links were broken or required manual intervention, this had to be solved with exceptions for different years.

We developed parsers for this matter and expect changes in the future that can be solved by adding simple exceptions to new years given that they change, which at this stage is uncertain.

For the scraping part of the project we used **R**'s **rvest** library.


## Wrangling

The reports published by the Ministry of Health are given in **excel** format. **R**'s **tidiyverse** and **readxl** libraries were used.

These reports are given in multi sheet excel files that have changed considerably over the years and had to be solved with individual parsers.

For this we created dataframes that contained metadata for each excel file including: Name of the sheet, cell range, format, and separators. 

3 different parsers were developed for each 3 of the datasets we created.
An extra parser was used for the 2010 dataframe asi it changed considerabily and required manual intervention.


Information is exported in Long format for ease of use and readability.


# Achievements

We managed to create 3 datasets that aggregate the information in NZ Ministry of Health's Mental Health annual reports.

# Acknowledgements

We wouldn't be here without the help of others. If you owe any attributions or thanks, include them here along with any citations of past research.

Main Contributors: Damian Sastre, Menglong Shi.

We would like to thank Thomas Li and Giulio Dalla Riva from the University of Canterbury for helping us wrangle this data.

